<a href="https://colab.research.google.com/github/Danieljoshua720/Electricity-demand-forecasting-AIML/blob/main/AIML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Attention
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
df=pd.read_csv('/content/powerdemand_5min_2021_to_2024_with weather.csv')

In [3]:
df.isnull().sum()

,0
Unnamed: 0,0
datetime,0
Power demand,0
temp,0
dwpt,0
rhum,0
wdir,540
wspd,0
pres,0
year,0


In [4]:
df.head()

,Unnamed: 0,datetime,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
0,0,2021-01-01 00:30:00,2014.00,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1,0,30,NaN
1,1,2021-01-01 00:35:00,2005.63,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1,0,35,NaN
2,2,2021-01-01 00:40:00,1977.60,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1,0,40,1999.076667
3,3,2021-01-01 00:45:00,1976.44,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1,0,45,1986.556667
4,4,2021-01-01 00:50:00,1954.37,8.0,6.9,93.0,0.0,0.0,1017.0,2021,1,1,0,50,1969.470000


In [5]:
df = df.drop(columns=['Unnamed: 0'])


In [6]:

num_cols = ['wdir', 'pres', 'year', 'month', 'day', 'hour', 'minute', 'moving_avg_3']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())


In [7]:
df.isnull().sum()

,0
datetime,0
Power demand,0
temp,0
dwpt,0
rhum,0
wdir,0
wspd,0
pres,0
year,0
month,0


In [8]:
df['datetime'] = pd.to_datetime(df['datetime'])


In [9]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393440 entries, 0 to 393439
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   datetime      393440 non-null  datetime64[ns]
 1   Power demand  393440 non-null  float64       
 2   temp          393440 non-null  float64       
 3   dwpt          393440 non-null  float64       
 4   rhum          393440 non-null  float64       
 5   wdir          393440 non-null  float64       
 6   wspd          393440 non-null  float64       
 7   pres          393440 non-null  float64       
 8   year          393440 non-null  int32         
 9   month         393440 non-null  int32         
 10  day           393440 non-null  int32         
 11  hour          393440 non-null  int32         
 12  minute        393440 non-null  int32         
 13  moving_avg_3  393440 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int32(5)
memory usage: 34.5 MB


In [18]:
X = df.drop(columns=['datetime', 'Power demand'])
y = df[['Power demand']]


In [19]:
df.tail()

,datetime,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
393435,2024-12-12 00:10:00,2146.84,12.3,6.8,69.0,269.0,1.8,1019.4,2024,12,12,0,10,2174.893333
393436,2024-12-12 00:15:00,2116.66,12.3,6.8,69.0,269.0,1.8,1019.4,2024,12,12,0,15,2139.416667
393437,2024-12-12 00:20:00,2082.77,12.3,6.8,69.0,269.0,1.8,1019.4,2024,12,12,0,20,2115.423333
393438,2024-12-12 00:25:00,2059.17,12.3,6.8,69.0,269.0,1.8,1019.4,2024,12,12,0,25,2086.200000
393439,2024-12-12 00:30:00,2049.66,12.3,6.8,69.0,269.0,1.8,1019.4,2024,12,12,0,30,2063.866667


In [20]:
train_size = int(0.8 * len(df))
val_size   = int(0.1 * len(df))

X_train = X[:train_size]
X_val   = X[train_size:train_size + val_size]
X_test  = X[train_size + val_size:]

y_train = y[:train_size]
y_val   = y[train_size:train_size + val_size]
y_test  = y[train_size + val_size:]

In [21]:
from sklearn.preprocessing import MinMaxScaler

scaler_X = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled   = scaler_X.transform(X_val)
X_test_scaled  = scaler_X.transform(X_test)


In [22]:
scaler_y = MinMaxScaler(feature_range=(0, 1))

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)


In [28]:
import numpy as np

def create_sequences(X, y, time_steps=24):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i+time_steps])
        y_seq.append(y[i+time_steps])
    return np.array(X_seq), np.array(y_seq)


In [29]:
time_steps = 24  # you can adjust based on problem

X_train_lstm, y_train_lstm = create_sequences(X_train_scaled, y_train_scaled, time_steps)
X_val_lstm, y_val_lstm     = create_sequences(X_val_scaled, y_val_scaled, time_steps)
X_test_lstm, y_test_lstm   = create_sequences(X_test_scaled, y_test_scaled, time_steps)


In [30]:
print(X_train_lstm.shape)
print(y_train_lstm.shape)


(314728, 24, 12)
(314728, 1)


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 50)             │        12,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,651 (49.42 KB)

 Trainable params: 12,651 (49.42 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Conv1D, MaxPooling1D
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Dictionary to store results
results = {}

# -------------------
# 1️⃣ Vanilla LSTM
# -------------------
vanilla_lstm = Sequential([
    LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)
])
vanilla_lstm.compile(optimizer='adam', loss='mse')
vanilla_lstm.fit(X_train_lstm, y_train_lstm,
                  validation_data=(X_val_lstm, y_val_lstm),
                  epochs=10, batch_size=64, verbose=1)
y_pred_vanilla = vanilla_lstm.predict(X_test_lstm)
results['Vanilla'] = y_pred_vanilla

# -------------------
# 2️⃣ Stacked LSTM
# -------------------
stacked_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    LSTM(50),
    Dense(1)
])
stacked_lstm.compile(optimizer='adam', loss='mse')
stacked_lstm.fit(X_train_lstm, y_train_lstm,
                 validation_data=(X_val_lstm, y_val_lstm),
                 epochs=10, batch_size=64, verbose=1)
y_pred_stacked = stacked_lstm.predict(X_test_lstm)
results['Stacked'] = y_pred_stacked

# -------------------
# 3️⃣ LSTM + Dropout
# -------------------
lstm_dropout = Sequential([
    LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dropout(0.2),
    Dense(1)
])
lstm_dropout.compile(optimizer='adam', loss='mse')
lstm_dropout.fit(X_train_lstm, y_train_lstm,
                 validation_data=(X_val_lstm, y_val_lstm),
                 epochs=10, batch_size=64, verbose=1)
y_pred_dropout = lstm_dropout.predict(X_test_lstm)
results['Dropout'] = y_pred_dropout

# -------------------
# 4️⃣ Bidirectional LSTM
# -------------------
bidir_lstm = Sequential([
    Bidirectional(LSTM(50), input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)
])
bidir_lstm.compile(optimizer='adam', loss='mse')
bidir_lstm.fit(X_train_lstm, y_train_lstm,
               validation_data=(X_val_lstm, y_val_lstm),
               epochs=10, batch_size=64, verbose=1)
y_pred_bidir = bidir_lstm.predict(X_test_lstm)
results['Bidirectional'] = y_pred_bidir

# -------------------
# 5️⃣ CNN + LSTM
# -------------------
cnn_lstm = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    MaxPooling1D(pool_size=2),
    LSTM(50),
    Dense(1)
])
cnn_lstm.compile(optimizer='adam', loss='mse')
cnn_lstm.fit(X_train_lstm, y_train_lstm,
             validation_data=(X_val_lstm, y_val_lstm),
             epochs=10, batch_size=64, verbose=1)
y_pred_cnn = cnn_lstm.predict(X_test_lstm)
results['CNN-LSTM'] = y_pred_cnn


Epoch 1/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 67s 13ms/step - loss: 0.0056 - val_loss: 5.5655e-04
Epoch 2/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 68s 14ms/step - loss: 1.8990e-04 - val_loss: 2.2255e-04
Epoch 3/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 66s 13ms/step - loss: 1.6432e-04 - val_loss: 2.1081e-04
Epoch 4/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 81s 13ms/step - loss: 1.4578e-04 - val_loss: 1.7248e-04
Epoch 5/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 66s 13ms/step - loss: 1.2078e-04 - val_loss: 1.6925e-04
Epoch 6/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 81s 13ms/step - loss: 1.1858e-04 - val_loss: 1.6768e-04
Epoch 7/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 84s 14ms/step - loss: 1.1011e-04 - val_loss: 1.7881e-04
Epoch 8/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 67s 14ms/step - loss: 1.1894e-04 - val_loss: 1.8501e-04
Epoch 9/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 80s 13ms/step - loss: 1.0777e-04 - val_loss: 1.6586e-04
Epoch 10/10
4918/4918 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - loss: 1.0847e-04 - val_loss: 1.9205e-04
1229/1229 ━━━━━━━━━━━━━